# Introduction

데이터는 유저와 광고의 features로 되어 있음 (categorical, binary, numerical)  
유저가 광고를 보고 클릭은 안했지만, 다이렉트로 앱을 설치한 경우도 있음.  

train/test 데이터를 만들기 위해. 총 22일간 데이터가 있음.  
첫 21일 간의 데이터는 train으로 사용하고, 22번째 날이 test로 사용되는 것임.  
train에는 impression record가 있음. 이는 유저가 광고를 보면서 생기게 되는 것임.. (클릭과 무관하게 광고를 봄으로 인해서 생기는 feature인듯)  

is_clicked, is_installed는 binary하게 들어가고 4개의 조합이 생김.
뭐, 클릭하고 설치하고(1,1), 클릭 안하고 설치하고(0,1) 이런식으로  
데이터를 관리 가능한 사이즈로 유지하고, 클릭 수, 변환율과 같은 기밀 정보를 보호하기 위해 4개의 조합에 해당하는 레코드를 차등적으로 다운 샘플링함

**TASK**  
is_clicked, is_installed 이 label 이고 예측하는 거임.

**데이터 설명**  
train은 총 30개 파일로 쪼개져있음. test는 1개

**train 설명**  
각 row는 impression record임.  
각 파일은 다음과 같은 형식으로 이루어짐

1. 첫번째 행은 헤더. 80개의 feature와 2개의 label  
   a. f_0은 rowId  
   b. f_1은 Date  
   c. Categorical feature (f_2 ~ f_32) => 31개   
   d. Binary feature (f_33 ~ f_41)  => 9개  
   e. Numerical feature (f_42 ~ f_79)  => 38개  
2. feature 몇개는 nullable하다. 이는 빈 문자열로 표시됌.  

**test 설명**  
걍 label만 없음.

**제출 형태**  
RowId | is_clicked | is_installed

In [ ]:
import pandas as pd 
import os

## Data Load

In [2]:
train_path = "./train"
test_path = "./test"

train_files = os.listdir(train_path)
test_files = os.listdir(test_path)

print(len(train_files), len(test_path))

30 6


In [3]:
def GetAllDataList(t="train"):
    total = []
    
    if t == "train":
        files = train_files
        path = train_path
    elif t == "test":
        files = test_files
        path = test_path
    else:
        return False
        
    for file_name in files:
        total.append(pd.read_csv(os.path.join(path, file_name), sep="\t"))
        
    return total

In [4]:
total_train = GetAllDataList(t="train")
total_test = GetAllDataList(t="test")

### train data shape

In [5]:
# 총 train row 갯수
# 348만개

total_train_rows = 0

for t in total_train:
    total_train_rows += t.shape[0]
    
total_train_rows

3485852

In [6]:
total_train[0].shape

(116474, 82)

In [7]:
total_train[0].head()

,f_0,f_1,f_2,f_3,f_4,f_5,f_6,f_7,f_8,f_9,...,f_72,f_73,f_74,f_75,f_76,f_77,f_78,f_79,is_clicked,is_installed
0,2541162,51,6581,22294,5491,25604,14597,27941,18800,31372,...,3.997850,1.713364,0.115692,1.156922,0.269948,0.0,0.0,0.0,0,0
1,2541260,49,20095,563,22861,15908,22651,27941,21218,869,...,0.000000,0.000000,0.115692,1.156922,0.269948,0.0,0.0,0.0,0,0
2,2541318,51,20095,563,22861,25604,22651,27941,21218,31372,...,0.000000,0.000000,0.038564,1.156922,0.269948,0.0,0.0,0.0,0,0
3,2541770,64,13658,22294,7003,25604,29975,27941,21218,21533,...,1.713364,1.142243,0.115692,1.156922,0.269948,0.0,0.0,0.0,0,0
4,2542002,62,20095,563,31686,15908,590,27941,18800,23218,...,0.000000,0.000000,0.115692,1.156922,0.269948,0.0,0.0,0.0,0,0


In [52]:
len(total_train[6]["f_37"].unique())

2

In [9]:
total_train[0].describe()

,f_0,f_1,f_2,f_3,f_4,f_5,f_6,f_7,f_8,f_9,...,f_72,f_73,f_74,f_75,f_76,f_77,f_78,f_79,is_clicked,is_installed
count,1.164740e+05,116474.000000,116474.000000,116474.000000,116474.000000,116474.000000,116474.000000,116474.0,116474.000000,116474.000000,...,116474.000000,116474.000000,116474.000000,116474.000000,116474.000000,116474.000000,116474.000000,116474.000000,116474.000000,116474.000000
mean,1.744250e+06,54.879321,18419.332658,13429.314843,18737.625358,22426.688445,16298.813830,27941.0,19987.323102,15042.630115,...,0.854000,0.247589,0.064929,1.002405,0.224727,0.071576,1.075889,0.359807,0.221895,0.175370
std,1.006799e+06,6.340613,8327.696053,9853.647952,9439.585121,3359.990192,10280.704814,0.0,1055.055932,9605.018446,...,1.287339,0.557335,0.054861,0.355751,0.094464,1.670550,7.177732,3.832728,0.415523,0.380285
min,1.900000e+01,45.000000,788.000000,563.000000,61.000000,15908.000000,20.000000,27941.0,18800.000000,869.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,8.704210e+05,49.000000,12412.000000,563.000000,11299.000000,21545.000000,6707.000000,27941.0,19203.000000,6675.000000,...,0.000000,0.000000,0.000000,1.156922,0.269948,0.000000,0.000000,0.000000,0.000000,0.000000
50%,1.747697e+06,55.000000,20095.000000,22294.000000,18971.000000,21545.000000,18566.000000,27941.0,19203.000000,14659.000000,...,0.571121,0.000000,0.115692,1.156922,0.269948,0.000000,0.000000,0.000000,0.000000,0.000000
75%,2.616234e+06,60.000000,23611.000000,22294.000000,26124.000000,25604.000000,23841.000000,27941.0,21218.000000,23218.000000,...,1.142243,0.571121,0.115692,1.156922,0.269948,0.000000,0.000000,0.000000,0.000000,0.000000
max,3.485842e+06,66.000000,32683.000000,22294.000000,32745.000000,29304.000000,32762.000000,27941.0,21621.000000,31372.000000,...,5.711215,3.997850,0.115692,1.156922,0.269948,74.769150,485.999477,186.922876,1.000000,1.000000


### test data shape

In [10]:
# 총 test row 갯수
# 16만개

total_test_rows = 0

for t in total_test:
    total_test_rows += t.shape[0]
    
total_test_rows

160973

In [11]:
total_test[0].shape

(160973, 80)

In [12]:
total_test[0].head()

,f_0,f_1,f_2,f_3,f_4,f_5,f_6,f_7,f_8,f_9,...,f_70,f_71,f_72,f_73,f_74,f_75,f_76,f_77,f_78,f_79
0,64505,67,26325,7152,21563,19475,31440,27941,21621,14659,...,1.519085,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0
1,64506,67,20095,563,22861,19475,21280,27941,19203,14659,...,0.000000,0.0,0.000000,0.0,0.115692,1.156922,0.269948,0.0,0.0,0.0
2,64507,67,890,22294,18294,21545,20210,27941,18800,9638,...,0.982995,0.0,0.000000,0.0,0.000000,1.156922,0.269948,0.0,0.0,0.0
3,64508,67,20095,563,22861,25604,21280,27941,18800,14659,...,0.000000,0.0,2.284486,0.0,0.000000,1.156922,0.269948,0.0,0.0,0.0
4,64509,67,27426,22294,11338,19475,23855,27941,21218,9638,...,0.986040,0.0,0.000000,0.0,0.077128,0.077128,0.077128,0.0,0.0,0.0


In [13]:
len(total_test[0]["f_0"].unique())

160973